## Closest Pair Implementations

**Corinna Pilcher**

A closest pair function returns a closest pair of points (if there is more than one set of equidistant points it will only return one). This notebook includes brute force and divide and conquer methods. Both will use the below `distance` method, which calculates the distance between two points using the Pythagorean Theorem.

In [1]:
from typing import List, Tuple
from math import inf, sqrt
Point = Tuple[float,float]  # Type alias for a point

In [2]:
def distance(p1: Point, p2: Point) -> float:
    return sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2) 

The `@timeit` decorator permits comparison of runtimes between these two closest pair implementations.

In [3]:
import time
def timeit(func):
    """
    A decorator that times the function func with the arguments that
    are passed to it
    :param func: Function being timed
    :return: a pair with func's result and the time it took to run. 
    """
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        return result, elapsed
    return clocked

#### Brute Force

The brute force technique involves comparing every point to every other point. This runs in $n^2$ time, where $n$ is the number of points in the `points` List.

In [4]:
@timeit
def brute(points: List[Point]) -> Tuple[Point,Point]:
    minDistance = inf
    n = len(points)

    # From point 0 through point n-2, since the last point will have already been compared to all others
    for i in range(n - 1):

        # From the point after this one at i through all others (points before i have already been compared to i)
        for j in range(i + 1, n):
            
            # Calculate distance between points
            pi = points[i]
            pj = points[j]
            d = distance(pi, pj)
            
            # Update output if this is the new min distance
            if d < minDistance:
                minDistance = d
                p1 = pi
                p2 = pj

    return p1, p2

#### Divide and Conquer

The divide and conquer method, `closest`, splits the list and uses brute force only on groups of three or fewer points, avoiding the $n^2$ runtime. Solutions are combined by comparing the smallest distance from each sublist, then checking for closer points across the splits.

The divide and conquer method depends on an adaptation of the former `brute` method, returning distance in addition to the original return value of the pair of closest points, supporting the comparison of returned values.

In [5]:
def brute_dist(points: List[Point]) -> Tuple[Point,Point] and int:
    minDistance = inf
    p1, p2 = (0, 0), (0, 0)
    n = len(points)

    # From point 0 through point n-2, since the last point will have already been compared to all others
    for i in range(len(points) - 1):
        
        # From the point after this one at i through all others (points before i have already been compared to i)
        for j in range(i+1, len(points)):
            
            # Calculate distance between points
            pi = points[i]
            pj = points[j]
            d = distance(pi, pj)
            
            # Check if this is the new min distance
            if d < minDistance:
                minDistance = d
                p1 = pi
                p2 = pj
    
    return (p1, p2), minDistance

`closest` also uses simple methods to intuitively extract x- and y-coordinates of a Point.

In [6]:
# Returns the x-coordinate of a point
def getX(point: Point) -> float:
    return point[0]

# Returns the y-coordinate of a point
def getY(point: Point) -> float:
    return point[1]

In [7]:
@timeit
def closest(points: List[Point]) -> Tuple[Point,Point]: 

    # Too few points to find a closest pair
    if len(points) < 2:
        return None

    # Recursive function
    def dac(pts: List[Point]) -> Tuple[Point,Point] and int:
        numberPoints = len(pts)
           
        # BASE CASE--------------------------------------------------------
        # Group of no more than 3 points, find closest pair via brute force
        if numberPoints <= 3:
            return brute_dist(pts)
        
        # LEFT-RIGHT DIVIDE------------------------------------------------
        # numberPoints / 2 to find the middle index
        midIndex = numberPoints >> 1 
        
        # Find the middle value of x to define midline for later use 
        x1=pts[midIndex][0]
        x2=pts[midIndex + 1][0]
        midline = (x1+x2) / 2
        
        # Find the closest pair in each half
        (p_left1, p_left2), d_left = dac(pts[:midIndex])
        (p_right1, p_right2), d_right = dac(pts[midIndex:])
        
        # LEFT-RIGHT COMBINATION-------------------------------------------
        # Combine solutions by finding the smallest distance between R and L sections
        # If left pair is closer together
        if d_left < d_right:
            d = d_left
            (p1, p2) = p_left1, p_left2
            
        # If right pair is closer together
        else:
            d = d_right
            (p1, p2) = p_right1, p_right2
        
 
        # CHECK ACROSS MIDLINE---------------------------------------------
        # Check for closer pairs across the midline
        
        # Create subset of points that could be closer by searching d before and d after midline
        startMiddle = midline - d
        endMiddle = midline + d
        midpoints = [xy for xy in pts if (getX(xy) > startMiddle) & (getX(xy) > endMiddle)]
        
        # Sort subset by y-coordinate - this is a copy of the list
        midpoints = sorted(midpoints, key=getY) 
        
        # Check the seven closest points to each midpoint
        # For each midpoint
        for i in range(len(midpoints)):
            
            # Check the seven closest points
            for j in range(7):
                
                # If index is valid
                if((i + 1 + j) < len(midpoints)):
                    p_mid1 = midpoints[i]
                    p_mid2 = midpoints[i + 1 + j]
                    d_mid = distance(p_mid1, p_mid2)
                    
                    # COMBINE LEFT-RIGHT AND MIDDLE SOLUTIONS--------------
                    # Find ultimate smallest distance between L/R min distance and mid distance
                    if(d_mid < d):
                        d = d_mid
                        p1 = p_mid1
                        p2 = p_mid2
                    
    
        # RETURN CLOSEST PAIR----------------------------------------------
        return (p1, p2), d
        
    # Sort points by x-coordinates, but only once
    points = sorted(points, key=getX) 
    
    # Call the recursive function to find the closest pair via DAC
    (p1, p2), d = dac(points)
    
    return (p1, p2)

Here's a quick time comparison test:

In [8]:
from random import random as rand, seed
seed(42)

In [9]:
n = 1024
pts = [(round(rand()*1000,3),round(rand()*1000,3)) for i in range(n)]

brute_pts, brute_time = brute(pts)
closest_pts, closest_time = closest(pts)

print("BRUTE --->", brute_time, "seconds to find", brute_pts, "as the closest pair.")
print("CLOSEST --->", closest_time, "seconds to find", closest_pts, "as the closest pair.")

BRUTE ---> 0.9900552000000005 seconds to find ((846.104, 92.298), (847.133, 92.464)) as the closest pair.
CLOSEST ---> 0.046936899999998616 seconds to find ((846.104, 92.298), (847.133, 92.464)) as the closest pair.


Closest is much more efficient, especially when it comes to larger input size ($n$).